In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
import warnings

cf10

In [ ]:
df_path_cf10 = 'cf10-rn/ww-df'

In [ ]:
def delta_merge_layers(folder_path,suffix):
    # Initialize dictionaries to store DataFrames for ft and non-ft files
    dataframes_ft = {}
    dataframes_non_ft = {}

    # List all CSV files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Loop through each file and categorize it as 'ft' or 'non-ft'
    for file in files:
        # Check if file matches the ft pattern
        if f'_{suffix}_ft.csv' in file:
            base_name = file.replace(f'_{suffix}_ft.csv', '')
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_ft[base_name] = df
        # Check if file matches the non-ft pattern
        elif f'_{suffix}.csv' in file:
            base_name = file.replace(f'_{suffix}.csv', '')
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the non-ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_non_ft[base_name] = df

    # Initialize a list to store the differences DataFrames
    list_differences = []

    # Compute the differences for each matching pair
    for base_name in dataframes_ft:
        if base_name in dataframes_non_ft:
            df_ft = dataframes_ft[base_name]
            df_non_ft = dataframes_non_ft[base_name]

            # Align DataFrames on both axes (rows and columns)
            df_ft_aligned, df_non_ft_aligned = df_ft.align(df_non_ft, join='outer', axis=None, fill_value=0)

            # Convert boolean columns to integers
            for df in [df_ft_aligned, df_non_ft_aligned]:
                bool_cols = df.select_dtypes(include=['bool']).columns
                df[bool_cols] = df[bool_cols].astype(int)

            # Convert all columns to numeric, coercing errors to NaN
            df_ft_numeric = df_ft_aligned.apply(pd.to_numeric, errors='coerce')
            df_non_ft_numeric = df_non_ft_aligned.apply(pd.to_numeric, errors='coerce')

            # Fill NaN values with 0 (optional, depending on how you want to handle missing values)
            df_ft_numeric = df_ft_numeric.fillna(0)
            df_non_ft_numeric = df_non_ft_numeric.fillna(0)

            # Compute the difference between the ft and non-ft DataFrames
            df_diff = df_ft_numeric - df_non_ft_numeric

            # Add a column to identify the pair
            df_diff['base_name'] = base_name
            df_diff['layer_id'] = df_ft_numeric['layer_id']

            # Optional: Reset index to ensure a proper stacking
            df_diff.reset_index(drop=True, inplace=True)

            # Append to the list
            list_differences.append(df_diff)

    # Concatenate all the difference DataFrames
    if list_differences:
        df_all_differences = pd.concat(list_differences, ignore_index=True)
    else:
        df_all_differences = pd.DataFrame()  # Return empty DataFrame if no differences found

    return df_all_differences


In [ ]:
la_cf10 = delta_merge_layers(df_path_cf10, 'fl')

In [ ]:
la_cf10.shape

In [ ]:
la_cf10 = la_cf10[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]

In [ ]:
la_d_cf10 = la_cf10.drop(la_cf10[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_cf10[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

In [ ]:
def compute_pairwise_differences(df):
    """
    Computes pairwise differences between all categories and 'bs' based on 'base_name',
    for matching 'layer_id's.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing 'base_name', 'layer_id', and numeric columns.

    Returns:
        pd.DataFrame: A DataFrame containing the differences with columns 'base_name', 'layer_id', and difference columns.
    """
    # Identify numeric columns excluding 'layer_id'
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    if 'layer_id' in numeric_cols:
        numeric_cols.remove('layer_id')

    # Separate 'bs' and other categories
    df_bs = df[df['base_name'] == 'bs']
    df_others = df[df['base_name'] != 'bs']

    # Merge on 'layer_id'
    df_merged = pd.merge(
        df_others,
        df_bs,
        on='layer_id',
        suffixes=('', '_bs'),
        how='inner'  # Ensure only matching 'layer_id's are joined
    )

    # Compute differences for numeric columns
    for col in numeric_cols:
        df_merged[f'{col}_diff'] = df_merged[col] - df_merged[f'{col}_bs']

    # Prepare the final DataFrame
    cols_to_keep = ['base_name', 'layer_id'] + [f'{col}_diff' for col in numeric_cols]
    df_differences = df_merged[cols_to_keep]

    # Reset index and sort (optional)
    df_differences.reset_index(drop=True, inplace=True)
    df_differences.sort_values(by=['base_name', 'layer_id'], inplace=True)

    return df_differences

In [ ]:
la_dif_cf10 = compute_pairwise_differences(la_d_cf10)

In [ ]:
def categorize_index(index_value):
    if 'div' in index_value:
        return 'div'
    elif 'dp' in index_value:
        return 'dp'
    elif 'wd' in index_value:
        return 'wd'
    else:
        return 'bs'  # Optional: in case none of the patterns match

# Apply the function to the index and create a new column
la_dif_cf10['category'] = la_dif_cf10['base_name'].map(categorize_index)

In [ ]:
la_avg_cf10 = la_dif_cf10.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_cf10

In [ ]:
la_avg_cf10.loc[la_avg_cf10['layer_id'] == 343, 'layer_id'] = 235

plot

In [ ]:
la_avg_cf10.groupby('category').mean()

cf100

In [ ]:
df_path_cf100 = 'cf100-rn/ww-df'
la_cf100 = delta_merge_layers(df_path_cf100, 'fl')

In [ ]:
la_cf100 = la_cf100[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_cf100

In [ ]:
la_d_cf100 = la_cf100.drop(la_cf100[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_cf100[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

In [ ]:
la_dif_cf100 = compute_pairwise_differences(la_d_cf100)
la_dif_cf100['category'] = la_dif_cf100['base_name'].map(categorize_index)

In [ ]:
la_avg_cf100 = la_dif_cf100.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_cf100

In [ ]:
la_avg_cf100.loc[la_avg_cf100['layer_id'] == 343, 'layer_id'] = 235

In [ ]:
la_avg_cf100.groupby('category').mean()

cars

In [ ]:
df_path_car = 'car_train/ww-df'
la_car = delta_merge_layers(df_path_car, 'int')

la_car = la_car[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_car

In [ ]:
la_d_car = la_car.drop(la_car[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_car[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

la_dif_car = compute_pairwise_differences(la_d_car)
la_dif_car['category'] = la_dif_car['base_name'].map(categorize_index)

la_dif_car


In [ ]:
la_avg_car = la_dif_car.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_car

In [ ]:
la_avg_car.loc[la_avg_car['layer_id'] == 343, 'layer_id'] = 235

In [ ]:
la_avg_car.groupby('category').mean()

Dom

In [ ]:
def delta_merge_layers_(folder_path):
    # Initialize dictionaries to store DataFrames for ft and non-ft files
    dataframes_ft = {}
    dataframes_non_ft = {}

    # List all CSV files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Loop through each file and categorize it as 'ft' or 'non-ft'
    for file in files:
        # Check if file matches the ft pattern
        if file.startswith('ft_'):  # Identify "ft" files
            base_name = file.replace('ft_', '').replace('.csv', '')  # Extract base name
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_ft[base_name] = df
        # Check if file matches the non-ft pattern
        else:  # Identify "non-ft" files
            base_name = file.replace('.csv', '')  # Extract base name
            file_path = os.path.join(folder_path, file)
            # Read the entire DataFrame from the non-ft file
            df = pd.read_csv(file_path)
            if not df.empty:
                dataframes_non_ft[base_name] = df

    # Initialize a list to store the differences DataFrames
    list_differences = []

    # Compute the differences for each matching pair
    for base_name in dataframes_ft:
        if base_name in dataframes_non_ft:
            df_ft = dataframes_ft[base_name]
            df_non_ft = dataframes_non_ft[base_name]

            # Align DataFrames on both axes (rows and columns)
            df_ft_aligned, df_non_ft_aligned = df_ft.align(df_non_ft, join='outer', axis=None, fill_value=0)

            # Convert boolean columns to integers
            for df in [df_ft_aligned, df_non_ft_aligned]:
                bool_cols = df.select_dtypes(include=['bool']).columns
                df[bool_cols] = df[bool_cols].astype(int)

            # Convert all columns to numeric, coercing errors to NaN
            df_ft_numeric = df_ft_aligned.apply(pd.to_numeric, errors='coerce')
            df_non_ft_numeric = df_non_ft_aligned.apply(pd.to_numeric, errors='coerce')

            # Fill NaN values with 0 (optional, depending on how you want to handle missing values)
            df_ft_numeric = df_ft_numeric.fillna(0)
            df_non_ft_numeric = df_non_ft_numeric.fillna(0)

            # Compute the difference between the ft and non-ft DataFrames
            df_diff = df_ft_numeric - df_non_ft_numeric

            # Add a column to identify the pair
            df_diff['base_name'] = base_name
            df_diff['layer_id'] = df_ft_numeric['layer_id']

            # Optional: Reset index to ensure a proper stacking
            df_diff.reset_index(drop=True, inplace=True)

            # Append to the list
            list_differences.append(df_diff)

    # Concatenate all the difference DataFrames
    if list_differences:
        df_all_differences = pd.concat(list_differences, ignore_index=True)
    else:
        df_all_differences = pd.DataFrame()  # Return empty DataFrame if no differences found

    return df_all_differences


In [ ]:
df_path_dom = 'domainnet/ww-df'
la_dom = delta_merge_layers_(df_path_dom)

la_dom = la_dom[['layer_id', 'alpha',  'entropy', 'log_norm', 'log_spectral_norm', 'base_name']]
la_dom

In [ ]:
la_d_dom = la_dom.drop(la_dom[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']][(la_dom[['alpha',  'entropy', 'log_norm', 'log_spectral_norm']] == 0).all(axis=1)].index)

la_dif_dom = compute_pairwise_differences(la_d_dom)
la_dif_dom['category'] = la_dif_dom['base_name'].map(categorize_index)


la_avg_dom = la_dif_dom.drop(['base_name'], axis=1).groupby(['layer_id', 'category'], as_index=False).mean()
la_avg_dom

In [ ]:
la_avg_dom.loc[la_avg_dom['layer_id'] == 343, 'layer_id'] = 235

In [ ]:
la_avg_dom.groupby('category').mean()

Cross ALL

In [ ]:
la_all = (la_avg_cf10.iloc[:, 2:] + la_avg_cf100.iloc[:, 2:] + la_avg_car.iloc[:, 2:] + la_avg_dom.iloc[:, 2:])/4

In [ ]:
la_all['layer_id'] = la_avg_cf10['layer_id']
la_all['category'] = la_avg_cf10['category']

In [ ]:
la_all.iloc[:, [0, 1, 2, 3, 5]].groupby('category').mean()

In [ ]:
group_means = la_all.iloc[:, [0, 1, 2, 3, 5]].groupby('category').mean()
group_means_dict = group_means.to_dict(orient='list')
group_means_dict

In [ ]:
# Create the DataFrame
data = {'alpha_diff': [0.07224881190014353,
  0.08123956139115088,
  0.019839655821132066],
 'entropy_diff': [1.505595603814941e-05,
  1.313548446581723e-05,
  3.005355047169503e-06],
 'log_norm_diff': [-5.7894856513136994e-06,
  -3.315263957927679e-05,
  -0.008005103718949726],
 'log_spectral_norm_diff': [-0.0004998588777363196,
  -0.00042169980104127226,
  -0.008851174659530765]}
df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()


In [ ]:
# last layer 
la_all[la_all['layer_id']==235].iloc[:,:-2].to_dict(orient='list')

In [ ]:
# Create the DataFrame
data = {'alpha_diff': [4.151516333761567, 4.627669975169038, 2.2683777934609335],
 'entropy_diff': [2.7384831152787997e-05,
  6.778142394284897e-05,
  1.5237653998167084e-05],
 'log_norm_diff': [-0.0026347047471573684,
  -0.002359189500555317,
  -0.0036596441562315126],
 'log_spectral_norm_diff': [-0.0008062987978322042,
  -0.0035965202221269043,
  -0.004208156624767036]}

df = pd.DataFrame(data, index=['div', 'dp', 'wd'])

# Set up subplots: 1 row, 5 columns
fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharey=False)

# Plot each metric separately
for i, col in enumerate(df.columns):
    ax = axes[i]
    colors = ['darkblue' if val < 0 else 'skyblue' for val in df[col]]
    df[col].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    ax.set_title(col, fontsize=12)
    ax.set_xticks(range(len(df.index)))
    ax.set_xticklabels(df.index, rotation=45, fontsize=10)
    ax.tick_params(axis='y', labelsize=9)
    ax.grid(axis='y', linestyle='--', alpha=0.5)


plt.tight_layout()  

plt.show()

In [ ]:
variables = ['alpha_diff', 'entropy_diff', 'log_norm_diff', 'log_spectral_norm_diff']

# Pivot the data to have categories as columns
pivot_df = la_all.pivot(index='layer_id', columns='category', values=variables)

# Compute Pearson correlation between categories for each variable
for var in variables:
    corr_matrix = pivot_df[var].corr(method='pearson')
    print(f'Pearson Correlation for {var}:\n{corr_matrix}\n')


In [ ]:
import itertools
from scipy.stats import f_oneway, ttest_ind

categories = ['div', 'dp', 'wd']

for var in variables:
    print(f'Variable: {var}')
    # Create a dictionary to hold series data for each category
    series_dict = {}
    for cat in categories:
        series = pivot_df[var][cat].dropna().values
        series_dict[cat] = series
    
    # ANOVA
    f_stat, p_value = f_oneway(*[series_dict[cat] for cat in categories])
    print(f'ANOVA: F-statistic={f_stat:.4f}, p-value={p_value:.4f}')
    
    # Pairwise t-tests
    for cat1, cat2 in itertools.combinations(categories, 2):
        t_stat, p_value = ttest_ind(series_dict[cat1], series_dict[cat2])
        print(f't-test between {cat1} and {cat2}: t-statistic={t_stat:.4f}, p-value={p_value:.4f}')
    print('-' * 50)

In [ ]:
import re

raw_text = """Variable: alpha_diff
ANOVA: F-statistic=0.2424, p-value=0.7850
t-test between div and dp: t-statistic=-0.0839, p-value=0.9333
t-test between div and wd: t-statistic=0.6083, p-value=0.5442
t-test between dp and wd: t-statistic=0.6743, p-value=0.5015
--------------------------------------------------
Variable: entropy_diff
ANOVA: F-statistic=0.1575, p-value=0.8544
t-test between div and dp: t-statistic=0.3028, p-value=0.7626
t-test between div and wd: t-statistic=0.4324, p-value=0.6662
t-test between dp and wd: t-statistic=0.3628, p-value=0.7174
--------------------------------------------------
Variable: log_norm_diff
ANOVA: F-statistic=23.8089, p-value=0.0000
t-test between div and dp: t-statistic=0.4111, p-value=0.6818
t-test between div and wd: t-statistic=4.8896, p-value=0.0000
t-test between dp and wd: t-statistic=4.8733, p-value=0.0000
--------------------------------------------------
Variable: log_spectral_norm_diff
ANOVA: F-statistic=27.0700, p-value=0.0000
t-test between div and dp: t-statistic=-0.2538, p-value=0.8001
t-test between div and wd: t-statistic=5.2031, p-value=0.0000
t-test between dp and wd: t-statistic=5.2990, p-value=0.0000
--------------------------------------------------"""  

# Split by each "Variable:"
sections = [s.strip() for s in raw_text.strip().split("Variable:") if s.strip()]

titles = []

for section in sections:
    lines = section.splitlines()
    var_name = lines[0].strip()
    
    # Extract ANOVA values
    anova_match = re.search(r"F-statistic=([-\d.]+), p-value=([-\d.]+)", section)
    f_val = float(anova_match.group(1))
    p_val = float(anova_match.group(2))
    p_str = f"p<0.001" if p_val == 0 else f"p={p_val:.2f}"

    # Extract all t-tests
    t_lines = re.findall(r"t-test between (\w+) and (\w+): t-statistic=([-\d.]+), p-value=([-\d.]+)", section)
    
    t_strs = []
    for a, b, t_stat, p_val in t_lines:
        t_val = float(t_stat)
        p_val = float(p_val)
        p_fmt = "p<0.001" if p_val == 0 else f"p={p_val:.2f}"
        t_strs.append(f"t({a}-{b}): t={t_val:.2f}, {p_fmt}")
    
    title = f'{var_name} | ANOVA: F={f_val:.2f}, {p_str} | ' + ' | '.join(t_strs)
    titles.append(title)

# Print all titles
for t in titles:
    print(f'title = (\n    "{t}"\n)')


In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

# Main plot
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='alpha_diff',
    hue='category',
    style='category',
    s=100,
    alpha=0.8,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Alpha', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

# Legend
ax.legend(title='Category', loc='upper left', fontsize=12, title_fontsize=12)

# ✅ Use absolute size + custom bottom-middle anchor
axins = inset_axes(
    ax,
    width=4, height=2,  # inches
    bbox_to_anchor=(0.25, 0.5, 0.5, 0.5),  # (x0, y0, width, height) in axes coords
    bbox_transform=ax.transAxes,
    loc='lower left'
)

# Zoomed-in scatterplot
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='alpha_diff',
    hue='category',
    style='category',
    s=70,
    alpha=0.8,
    palette='Set2',
    ax=axins,
    legend=False
)

title = (
    "ANOVA: F=0.24, p=0.79 | t(div-dp): t=-0.08, p=0.93 | t(div-wd): t=0.61, p=0.54 | t(dp-wd): t=0.67, p=0.50"
)
plt.suptitle(title, fontsize=12)


# Zoom-in limits and line
axins.set_ylim(-0.08, 0.08)
axins.set_xlim(la_all['layer_id'].min()-5, la_all['layer_id'].max()+5)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Optional: mark_inset (comment if visually confusing at bottom)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

plt.tight_layout()


plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='entropy_diff',
    hue='category',
    style='category',
    s=100,
    alpha=0.8,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Entropy', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

# Legend
ax.legend(title='Category', loc='lower left', fontsize=12, title_fontsize=12)

# ✅ Use absolute size + custom bottom-middle anchor
axins = inset_axes(
    ax,
    width=4, height=2,  # inches
    bbox_to_anchor=(0.25, 0.01, 0.5, 0.5),  # (x0, y0, width, height) in axes coords
    bbox_transform=ax.transAxes,
    loc='upper left'
)

# Zoomed-in scatterplot
sns.scatterplot(
    data=la_all,
    x='layer_id',
    y='entropy_diff',
    hue='category',
    style='category',
    s=70,
    alpha=0.8,
    palette='Set2',
    ax=axins,
    legend=False
)

title = (
    "ANOVA: F=0.16, p=0.85 | t(div-dp): t=0.30, p=0.76 | t(div-wd): t=0.43, p=0.67 | t(dp-wd): t=0.36, p=0.72"
)
plt.suptitle(title, fontsize=12)

# Zoom-in limits and line
axins.set_ylim(-0.0001, 0.0001)
axins.set_xlim(la_all['layer_id'].min()-5, la_all['layer_id'].max()+5)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Optional: mark_inset (comment if visually confusing at bottom)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

plt.tight_layout()


plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Main line plot
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Log Frobenius Norm', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)
ax.legend(title='Category', loc='lower left', fontsize=12, title_fontsize=12)

# Inset axis for zoom-in
axins = inset_axes(
    ax,
    width=4, height=2,
    bbox_to_anchor=(0.25, 0.15, 0.3, 0.3),
    bbox_transform=ax.transAxes,
    loc='upper left'
)

# Zoomed-in line plot
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=axins,
    legend=False
)

# Zoom-in limits and formatting
axins.set_ylim(-0.0001, 0.0002)
axins.set_xlim(la_all['layer_id'].min() - 3, la_all['layer_id'].max() + 3)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Mark zoomed region on main plot
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

# Title and layout
title = (
    "ANOVA: F=23.81, p<0.001 | t(div-dp): t=0.41, p=0.68 | t(div-wd): t=4.89, p<0.001 | t(dp-wd): t=4.87, p<0.001"
)
plt.suptitle(title, fontsize=12)

plt.tight_layout()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_spectral_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=ax
)

# Plot formatting
ax.set_xlabel('Layer Index', fontsize=14)
ax.set_ylabel('Log Spectral Norm', fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(True)

# Legend
ax.legend(title='Category', loc='lower left', fontsize=12, title_fontsize=12)

# ✅ Use absolute size + custom bottom-middle anchor
axins = inset_axes(
    ax,
    width=4, height=2,  # inches
    bbox_to_anchor=(0.25, 0.15, 0.3, 0.3), # (x0, y0, width, height) in axes coords
    bbox_transform=ax.transAxes,
    loc='upper left'
)

# Zoomed-in scatterplot
sns.lineplot(
    data=la_all,
    x='layer_id',
    y='log_spectral_norm_diff',
    hue='category',
    style='category',
    markers=True,
    dashes=False,
    palette='Set2',
    ax=axins,
    legend=False
)

title = (
    "ANOVA: F=27.07, p<0.001 | t(div-dp): t=-0.25, p=0.80 | t(div-wd): t=5.20, p<0.001 | t(dp-wd): t=5.30, p<0.001"
)
plt.suptitle(title, fontsize=12)

# Zoom-in limits and line
axins.set_ylim(-0.003, 0.002)
axins.set_xlim(la_all['layer_id'].min()-5, la_all['layer_id'].max()+5)
axins.axhline(0, color='black', linestyle='--', linewidth=1)
axins.set_xticklabels([])
axins.set_yticklabels([])
axins.set_xlabel('')
axins.set_ylabel('')

# Optional: mark_inset (comment if visually confusing at bottom)
mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="gray")

plt.tight_layout()

plt.show()